In [10]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.externals import joblib

In [11]:
dataset1 = pd.read_csv('dataset1.csv')
dataset1.label.replace(-1,0,inplace=True)
dataset2 = pd.read_csv('dataset2.csv')
dataset2.label.replace(-1,0,inplace=True)
dataset3 = pd.read_csv('dataset3.csv')

In [12]:
print(dataset3.shape,dataset2.shape,dataset2.shape)

(113234, 57) (257126, 56) (257126, 56)


In [13]:
pd.set_option('display.max_columns',200)   
dataset1.head()

,user_id,discount_rate,distance,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,merchant_coupon_transfer_rate,coupon_rate,count_merchant,user_min_distance,user_max_distance,user_mean_distance,user_median_distance,buy_use_coupon,buy_total,coupon_received,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,buy_use_coupon_rate,user_coupon_transfer_rate,user_merchant_buy_total,user_merchant_received,user_merchant_buy_use_coupon,user_merchant_any,user_merchant_buy_common,user_merchant_coupon_transfer_rate,user_merchant_coupon_buy_rate,user_merchant_rate,user_merchant_common_buy_rate,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after,is_weekend,weekday1,weekday2,weekday3,weekday4,weekday5,weekday6,weekday7,label
0,1832624,0.900000,0.0,29,16,200.0,20.0,1,14962.0,14962.0,14962.0,-1.0,10.0,1.209798,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,0,0,0,1,0,0,0
1,163606,0.850000,10.0,21,8,200.0,30.0,1,787.0,787.0,787.0,-1.0,10.0,2.439644,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,0,0,1,0,0,0,0
2,4061024,0.900000,10.0,26,13,200.0,20.0,1,14962.0,14962.0,14962.0,-1.0,10.0,1.209798,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,1,0,0,0,0,0,0
3,106443,0.833333,-1.0,29,16,30.0,5.0,1,6553.0,6553.0,6553.0,-1.0,10.0,0.664123,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,0,0,0,1,0,0,0
4,114747,0.850000,9.0,26,13,200.0,30.0,1,787.0,787.0,787.0,-1.0,10.0,2.439644,1.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,11.0,11.0,11.0,13.4,1.0,25.0,1.0,1.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,1,0,0,0,0,0,0


In [14]:
dataset1.drop_duplicates(inplace=True)
dataset2.drop_duplicates(inplace=True)
dataset3.drop_duplicates(inplace=True)

In [15]:
#把前面两个数据集整合在一起
dataset12 = pd.concat([dataset1,dataset2],axis=0)

In [16]:
dataset1_y = dataset1.label
dataset1_x = dataset1.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)  
# 'day_gap_before','day_gap_after' cause overfitting, 0.77

In [17]:
dataset2_y = dataset2.label
dataset2_x = dataset2.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)

In [18]:
dataset12_y = dataset12.label
dataset12_x = dataset12.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)

In [90]:
dataset3_preds = dataset3[['user_id','coupon_id','date_received']]
dataset3_x = dataset3.drop(['user_id','coupon_id','date_received','day_gap_before','day_gap_after'],axis=1)
dataset3_preds.shape
dataset3_x.shape

(113234, 52)

In [111]:
#将空值用0填充
dataset1_x=dataset1_x.fillna(0)
dataset2_x=dataset2_x.fillna(0)
dataset12_x=dataset12_x.fillna(0)
dataset3_x=dataset3_x.fillna(0)
dataset3_x.shape

(113234, 52)

In [ ]:
##官方的用法

In [98]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

In [118]:
params = {'n_estimators': 100, 'max_depth': 5, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)

In [104]:
dataset12_y.shape

(386897,)

In [119]:
clf.fit(dataset1_x, dataset1_y) #训练
mse = mean_squared_error(dataset2_y, clf.predict(dataset2_x)) #预测并且计算MSE
mse

In [141]:
preds= clf.predict(dataset3_x)
preds.reshape(-1,1)
preds= pd.DataFrame(preds)
preds.head()

,0
0,0.029666
1,0.042592
2,0.036181
3,0.037701
4,0.029905


In [148]:
preds.to_csv('D:/天池/GBDT_label.csv',index=None)

In [149]:
preds.shape

(113234, 1)

In [70]:
trainSub=dataset12_x
validSub=dataset12_y

In [71]:
#trainSub, validSub = train_test_split(dataset12_x, test_size = 0.2, stratify = dataset12_y, random_state=100)
x_train, x_test, y_train, y_test = train_test_split(dataset12_x, dataset12_y) #默认是75%训练集
x_train.shape
x_test.shape

(96725, 52)

In [79]:
# 模型训练，使用GBDT算法
gbr = GradientBoostingClassifier(
                        n_estimators=1000, 
                        max_depth=5, 
                        min_samples_split=2, 
                        learning_rate=0.01)
gbr.fit(x_train, y_train.ravel())
#joblib.dump(gbr, 'train_model_result4.m')   # 保存模型

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.01, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=1000,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [80]:
#训练和验证
y_gbr = gbr.predict(x_train)
y_gbr1 = gbr.predict(x_test)
acc_train = gbr.score(x_train, y_train)
acc_test = gbr.score(x_test, y_test)
print(acc_train)
print(acc_test)
y_gbr1.shape

0.9334808320582275
0.9299870767640217


(96725,)

In [87]:
preds= gbr.predict(dataset3_x)
preds=preds.reshape(-1,1)
preds.shape

(113234, 1)

In [88]:
preds = pd.DataFrame(preds)
#preds.rename(columns={'0':'label'}, inplace = True)
preds.shape
preds.head()

,0
0,0
1,0
2,0
3,0
4,0


In [89]:
preds.to_csv("gbdt_label.csv",index=None,header=None)

In [91]:
dataset3_preds.to_csv("gbdt_dataset3.csv",index=None,header=None)